In [1]:
import pandas as pd

In [2]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool, CustomJS, Slider
from bokeh.models.widgets import Div, Paragraph, Select
from bokeh.events import Tap, DoubleTap
from bokeh.layouts import layout
from bokeh.io import output_notebook, push_notebook
from ipywidgets import interact

In [3]:
import preprocessing_data

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
df = preprocessing_data.read_and_preprocess_from_csv('data/csv/oving5/1395669706/Partner.csv')

STOREDSTRING1
----------
Index: 2017-02-22T19:54:23.000000000
Original:
 package objectstructures;

public class Partner {

	
}

Before:
 package objectstructures;

public class Partner {

	

Inserted:
 
After:
 
	
}

Start: 53
End: -6
55
0
Test: False
package objectstructures;

public class Partner {

	

	
}

----------
Index: 2017-02-22T20:01:09.000000000
Original:
 package objectstructures;

public class Partner {

	

	
}

Before:
 package objectstructures;

public class Partner {

	
Inserted:
 Partner() {
		
	}
After:
 
	
}

Start: 52
End: -6
58
17
Test: False
package objectstructures;

public class Partner {

	Partner() {
		
	}
	
}

----------
Index: 2017-02-22T20:01:21.000000000
Original:
 package objectstructures;

public class Partner {

	Partner() {
		
	}
	
}

Before:
 package objectstructures;

public class Partner {

	Partner(
Inserted:
 String name, 
After:
 ) {
		
	}
	
}

Start: 60
End: -15
74
13
Test: False
package objectstructures;

public class Partner {

	Partner(Strin

----------
Index: 2017-02-22T20:24:58.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		if (partner == null) {
			this.partner = null;
		} else if (this.partner != partner) {
			this.partner = partner;
			partner.partner = this;
		} 
		
	}
	
	public static void main(String[] args) {
		Partner p1 = new Partner("1");
		Partner p2 = new Partner("2");
		Partner p3 = new Partner("3");
		
		p1.setPartner(p2);
		p1.setPartner(null);
		
	}
	
	
}

Before:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String 

----------
Index: 2017-02-22T20:39:30.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		if (partner == null) {
			if (this.partner.partner == this) {
				this.partner.setPartner(null);
			}
			this.partner = null;
			
		} else if (this.partner != partner) {
			this.partner = partner;
			partner.partner = this;
		} 
		
	}
	
	public static void main(String[] args) {
		Partner p1 = new Partner("1");
		Partner p2 = new Partner("2");
		
		p1.setPartner(p2);
		System.out.println(p1.partner);
		p1.setPartner(null);

		System.out.println(p2.partner);
	}
	
	
}

Before:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	

----------
Index: 2017-02-22T20:53:38.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		if (partner == null) {
			if (this.partner != null) {
				this.partner.setPartner(null);
			}
			
			this.partner = null;
		} else if (this.partner != partner) {
			partner.setPartner(this);
		} else {
			this.partner = partner;
		}
	}
	
	public static void main(String[] args) {
		Partner p1 = new Partner("1");
		Partner p2 = new Partner("2");
		
		p1.setPartner(p2);
		System.out.println(p1.partner);
		p1.setPartner(null);

		System.out.println(p2.partner);
	}
	
	
}

Before:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	


----------
Index: 2017-02-22T21:15:17.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		
		if (this.partner != null) {

			this.partner.partner = null;
			this.partner = null;
			
		} else if (this != partner) {
			
			this.partner = partner;
			if (this.partner.partner != this) {
				partner.setPartner(this);
			}
			
		} else {
			throw new IllegalArgumentException("You cannot be your own partner.");
		}
	}
	
	public static void main(String[] args) {
		Partner p1 = new Partner("1");
		Partner p2 = new Partner("2");
		
		p1.setPartner(p2);
		System.out.println(p1.partner);
		p1.setPartner(null);

		System.out.println(p2.partner);
	}
	
	
}

Before:
 package obje

...

----------
Index: 2017-02-22T21:38:41.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		
		if (this != partner) {
			if (this.partner != null) {
				
				if (this.partner != partner) {
					partner.partner = this;
					this.partner = partner;
					this.partner.partner = null;
				} 
				
			} else {
				this.partner = partner;
				partner.partner = this;
			}
			
			
		} else {
			throw new IllegalArgumentException("You cannot be your own partner.");
		}
	}
	
	public static void main(String[] args) {
		Partner p1 = new Partner("1");
		Partner p2 = new Partner("2");
		Partner p3 = new Partner("3");
		
		p1.setPartner(p2);
		System.out.println(p1.partner);
		p1.se

----------
Index: 2017-02-22T21:48:32.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		
		if (this != partner) {
			if (this.partner != null) {
				
				if (this.partner != partner && partner != null) {
					this.partner.partner = null;
					partner.partner = this;
					this.partner = partner;
				} else if (partner == null) {
					this.partner.partner = null;
					this.partner = null;
				}
				
			} else {
				this.partner = partner;
				partner.partner = this;
			}
			
			
		} else {
			throw new IllegalArgumentException("You cannot be your own partner.");
		}
	}
	
	public static void main(String[] args) {
		Partner p1 = new Partner("1");
		Partner p2 = new 

----------
Index: 2017-02-22T21:59:36.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		
		if (this != partner) {
			if (partner == null && this.partner.partner == this) {
				this.partner = null;
				this.partner.setPartner(null);
				
				
				this.partner = null;
				this.partner.setPartner(null);
			}
		}
		
		if (this != partner) {
			if (this.partner != null) {
				
				if (this.partner != partner && partner != null) {
					
					this.partner.partner = null;
					partner.partner = this;
					this.partner = partner;
					
				} else if (partner == null) {
					this.partner.partner = null;
					this.partner = null;
				}
				
			} else {
				this.partner = p

----------
Index: 2017-02-22T22:14:01.000000000
Original:
 package objectstructures;

public class Partner {

	private String name;
	private Partner partner;
	
	Partner(String name) {
		this.name = name;
	}
	
	public String toString() {
		return this.name;
	}
	
	public String getName() {
		return this.name;
	}
	
	public Partner getPartner() {
		return this.partner;
	}
	
	public void setPartner(Partner partner) {
		
		if (partner != null) {
			if (this.partner == null) {
				this.partner = partner;
				partner.setPartner(this);
			} else {
				
			}
		} else if (partner == null && this.partner != null) {
			this.partner = null;
			this.partner
			
		}
		
		if (this != partner) {
			if (this.partner != null) {
				
				if (this.partner != partner && partner != null) {
					
					this.partner.partner = null;
					partner.partner = this;
					this.partner = partner;
					
				} else if (partner == null) {
					this.partner.partner = null;
					this.partner = null;
				}
				
			} else {
		

c:\users\herman\appdata\local\conda\conda\envs\jexercise\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
df = preprocessing_data.read_and_preprocess_from_csv('data/csv/oving5/0141126194/Card.csv')

STOREDSTRING1
This file has only been edited once, skipping.
STOREDSTRING2
----------
Index: 2017-02-23T22:59:29.000000000
Original:
 package objectstructures;

import java.util.Arrays;

public class CardDeck {
	
	private Card[] spadeCards;
	private Card[] heartCards;
	private Card[] diamondCards;
	private Card[] clubCards;
	private Card[] allCards;
	
	public CardDeck(int n) {
		if (n < 1 || n > 13) {
			throw new IllegalAccessError("There are only 13 different faces of such a card game.");
		}
		spadeCards = new Card[n];
		heartCards = new Card[n];
		diamondCards = new Card[n];
		clubCards = new Card[n];
		allCards = new Card[4 * n];
		for (int i = 0; i < n; i++) {
			spadeCards[i] = new Card('S', i + 1);
			heartCards[i] = new Card('H', i + 1);
			diamondCards[i] = new Card('D', i + 1);
			clubCards[i] = new Card('C', i + 1);
			
			allCards[i] = spadeCards[i];
			allCards[i + n] = heartCards[i];
			allCards[i + 2 * n] = diamondCards[i];
			allCards[i + 3 * n] = clubCards[i];
		}
	}


c:\users\herman\appdata\local\conda\conda\envs\jexercise\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df = preprocessing_data.get_df_from_csv('data/csv/oving5/1395669706/Partner.csv')

In [7]:
df['index_formatted'] = df.index.strftime("%d-%m %H:%M:%S")

In [10]:
div = Div(text="", width=720)

In [29]:
java_keywords = [
    "package", "import", "class", "public", "private", "protected", "static", "void", "return"
]

In [34]:
def add_flavor(source_code):
    tokens = source_code.split(" ")
    new_tokens = []
    for token in tokens:
        if token.replace("<br />", "").replace("&emsp;", "") in java_keywords:
            new_tokens.append("<span style='color: #9C27B0;'>{}</span>".format(token))
        else:
            new_tokens.append(token)
    return " ".join(new_tokens)

In [35]:
def show_source_code(x, y):
    x = round(x)
    y = round(y)
    date_time = pd.to_datetime(x, unit="ms")
    nearest_row = df.iloc[df.index.get_loc(date_time, method='nearest')]
    file_number = ''.join(filter(str.isdigit, sm_selected))
    source_code = nearest_row['SourceCode' + file_number]
    source_code = source_code.replace("\n", "<br />").replace("\t", "&emsp;")
    div.text = "<p style='font-size: 90%; line-height: 100%'>{}</p>".format(add_flavor(source_code))
    push_notebook()

In [12]:
def register_event():
    "Build a suitable CustomJS to get the selected x and y values"
    return CustomJS(code="""
        var x = cb_obj['x'];
        var y = cb_obj['y'];
        var sx = cb_obj['sx'];
        var sy = cb_obj['sy'];
        if (IPython.notebook.kernel !== undefined) {
            var kernel = IPython.notebook.kernel;
            cmd = "show_source_code(" + x + ", " + y + ")";
            kernel.execute(cmd, {}, {});
        }
    """)

In [20]:
sm_selected = "SizeMeasure3"

In [21]:
source = ColumnDataSource(df)

hover = HoverTool( tooltips="""
    <div>
        <div>
            <span style="font-size: 12px; color: #01579B;">Timestamp: @index_formatted</span><br />
            <span style="font-size: 10px; color: #03A9F4;">{0}: @{0}</span>
        </div>
    </div>
    """.format(sm_selected)
)

p = figure(x_axis_type="datetime", plot_width=900, plot_height=400, tools=[hover,"pan","wheel_zoom","box_zoom","reset","tap"],
           title="SizeMeasure and SourceCode")

p.line('index', sm_selected, line_width=2, source=source)

p.js_on_event(Tap, register_event())
p.js_on_event(DoubleTap, register_event())

In [32]:
plot_layout = layout([
    [p],
    [div]
])

In [36]:
show(plot_layout, notebook_handle=True)

In [24]:
show(p, notebook_handle=True)

In [13]:
show(div, notebook_handle=True)